In [8]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import numpy as np
from geopandas.tools import sjoin
import folium
from folium.plugins import MarkerCluster
from folium import IFrame
import shapely
from shapely.geometry import Point
import unicodedata
import pysal as ps
import branca.colormap as cm
import datetime
import seaborn as sns
%config InlineBackend.figure_format = 'retina'
import matplotlib.ticker as ticker

lon = 54.093409
lat = -2.89479

df1 = pd.read_csv('AUTOMATION_LEAVE.csv')

fp = 'LA_17.shp'
map_df = gpd.read_file(fp)
map_df = map_df.to_crs(epsg=3395)
map_df = map_df.loc[map_df['lad17cd'].str[:1] == 'E']
map_df = map_df.dissolve(by='lad17cd')
df1['lad17cd'] = df1['LA_CODE']
map_df = pd.merge(map_df,df1,how='right', on='lad17cd')
map_df = map_df.loc[~map_df['P_LEAVE'].isnull()]

map_df['P_LEAVE'] = np.round(map_df['P_LEAVE'], decimals = 1)
map_df['P_AUTOMATION'] = np.round(map_df['P_AUTOMATION'], decimals = 1)

poss_map = folium.Map([lon, lat], zoom_start = 6, tiles='cartodbpositron')
map_df['DUMMY'] = 1
map_df2 = map_df.dissolve(by='DUMMY')

feat1 = 'P_LEAVE'
label1 = 'Percentage Leave'
ft1 = feat1

feat2 = 'P_AUTOMATION'
label2 = 'Percentage Automation'
ft2 = feat2

c_min1 = 35
c_max1 = 70

cmap1 = cm.linear.viridis.scale(c_min1, c_max1) 

c_min2 = 35
c_max2 = 50

cmap2 = cm.linear.viridis.scale(c_min2, c_max2) 

folium.GeoJson(map_df, name = label1, style_function=lambda feature: { 
             'fillColor': cmap1(feature['properties'][ft1]),'fillOpacity' : 1, 'weight' : 0.8, 'color' : 'gray' },
               smooth_factor=2.0, show = False,
               tooltip=folium.features.GeoJsonTooltip(fields=['lad17nm', 'P_LEAVE', 'P_AUTOMATION'],aliases=['Local Authority','Percentage Leave', 'Percentage Automation'], labels=True, sticky=True)                          
              ).add_to(poss_map) 
               
folium.GeoJson(map_df, name = label2, style_function=lambda feature: { 
             'fillColor': cmap2(feature['properties'][ft2]),'fillOpacity' : 1, 'weight' : 0.8, 'color' : 'gray' },
               smooth_factor=2.0, show = True,
               tooltip=folium.features.GeoJsonTooltip(fields=['lad17nm', 'P_LEAVE', 'P_AUTOMATION'],aliases=['Local Authority','Percentage Leave', 'Percentage Automation'], labels=True, sticky=True)                          
              ).add_to(poss_map) 

cmap1.caption = label1 
cmap1.add_to(poss_map)
cmap2.caption = label2 
cmap2.add_to(poss_map)

folium.LayerControl().add_to(poss_map)
poss_map.save(outfile='LEAVE_AUTOMATION.html')